In [6]:
#python -m pip install --upgrade pip
#pip install requests
#pip install beautifulsoup4

import csv
import requests
from bs4 import BeautifulSoup
import os

In [10]:
url ="https://www.imdb.com/title/tt7678620/episodes/?ref_=tt_ov_epl"
hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
res = requests.get(url, headers=hdr)
res.raise_for_status()

soup = BeautifulSoup(res.text, "html.parser")
#print(soup)

In [11]:
season = soup.find_all("a", attrs={"data-testid": "tab-season-entry"})
nums_season = len(season)
text_season = []
for s in season:
    print(s)
    text_season.append(s.get_text())

print(f"총 시즌 수:{nums_season}")
print(f"시즌 타이틀:{text_season}")

<a aria-disabled="false" class="ipc-tab ipc-tab-link ipc-tab--on-base ipc-tab--active" data-testid="tab-season-entry" href="/title/tt7678620/episodes/?season=1" role="tab" tabindex="0">1</a>
<a aria-disabled="false" class="ipc-tab ipc-tab-link ipc-tab--on-base" data-testid="tab-season-entry" href="/title/tt7678620/episodes/?season=2" role="tab" tabindex="0">2</a>
<a aria-disabled="false" class="ipc-tab ipc-tab-link ipc-tab--on-base" data-testid="tab-season-entry" href="/title/tt7678620/episodes/?season=3" role="tab" tabindex="0">3</a>
<a aria-disabled="false" class="ipc-tab ipc-tab-link ipc-tab--on-base" data-testid="tab-season-entry" href="/title/tt7678620/episodes/?season=4" role="tab" tabindex="0">4</a>
<a aria-disabled="false" class="ipc-tab ipc-tab-link ipc-tab--on-base" data-testid="tab-season-entry" href="/title/tt7678620/episodes/?season=Unknown" role="tab" tabindex="0">Unknown</a>
총 시즌 수:5
시즌 타이틀:['1', '2', '3', '4', 'Unknown']


In [15]:
filename = "블루이.csv"
f = open(filename, mode="w", encoding="utf-8-sig", newline="")
writer = csv.writer(f)

attributes = ["시즌", "방영일", "제목", "리뷰수", "평점", '줄거리']
writer.writerow(attributes)

img_dir_name = "블루이이미지"
if not os.path.isdir(img_dir_name):
    os.mkdir(img_dir_name)
cur_dir = os.getcwd()
# os.mkdir(f'{cur_dir}/{img_dir_name}')

season_url = "https://www.imdb.com/title/tt7678620/episodes?season={}"
for season in text_season:
    url =season_url.format(season)      
    res = requests.get(url, headers=hdr)
    res.raise_for_status()

    soup = BeautifulSoup(res.text, "html.parser")
    episodes = soup.find_all("article", attrs={"class": "sc-282bae8e-1 ekbCxw episode-item-wrapper"})

    for episode in episodes:            
            title = episode.find("a", attrs={"class": "ipc-title-link-wrapper"}).get_text()
            date_exist = episode.find("span", attrs={"class": "sc-ccd6e31b-10 dYquTu"})
            if date_exist != None:
                date = date_exist.get_text()
            else:
                date = "방영일이 없습니다."
            
            rating_votecount_exist = episode.find("span", attrs={"class": "ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating"})
            if rating_votecount_exist != None:                
                rating_votecount = rating_votecount_exist.get_text()
                rating = rating_votecount[0:3]
                votecount = rating_votecount[8:11]
            else:
                rating = "0"
                votecount = "0"
            
            descr_exist = episode.find("div", attrs={"class": "ipc-html-content-inner-div"})
            if descr_exist != None:
                descr = descr_exist.get_text()
            else:
                descr = "줄거리가 없습니다."

            print(title, date, rating, votecount, descr)
            data_rows = [season, date, title, votecount, rating, descr]
            writer.writerow(data_rows)

            img = episode.img
            imgurl = img['src'];
            r = requests.get(imgurl, headers=hdr)
            with open(f'{cur_dir}/{img_dir_name}/{title[0:]}.jpg', "wb") as out:
                out.write(r.content)

f.close()

S1.E1 ∙ Magic Xylophone Sun, May 24, 2020 7.7 800 As Bluey and Bingo begin to squabble over their magic xylophone (that has the power to freeze their Dad in space and time) Dad seizes control and freezes Bluey, leaving Bingo as her only hope.
S1.E2 ∙ Hospital Mon, Oct 1, 2018 7.9 687 Doctor Bluey is needed for assistance when Dad gets a very curious x-ray from Nurse Bingo. It seems that he has a cat in his tummy, leaving Bluey no choice but for her to operate immediately.
S1.E3 ∙ Keepy Uppy Tue, Oct 2, 2018 8.2 670 Bluey, Bingo and Mum are enjoying playing Keepy-Uppy around the house with their last balloon. All is fine, until their Dad joins in, and soon, the game becomes the ultimate test of skill.
S1.E4 ∙ Daddy Robot Wed, Oct 3, 2018 7.9 599 Bluey and Bingo don't want to tidy the playroom, so they get their Dad (as Daddy Robot) to do it for them. But when Bluey takes things too far, Daddy Robot begins to malfunction with some disastrous results.
S1.E5 ∙ Shadowlands Thu, Oct 4, 2018 

In [14]:
list_target = [['tt1475582', '셜록'], ['tt0944947', '왕좌의 게임']]
# 다른 드라마 정보 가져와 보기 (각 드라마마다 원하는 데이터를 가리키는 이름이 다를 수 있음)